# NWIS Flow Duration Curves

This notebook demonstrates how retrieve data from NWIS and use it to estimate flow duration curves. It borrows heavily from the work done by Paul Inkenbrandt and published here: http://earthpy.org/flow.htmlb

In [ ]:
import datetime as dt
import geopandas as gpd
import ulmo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sp
import scipy.optimize as op
import plotly.graph_objects as go

## Functions

The function `download_usgs_site_data` uses the <a href=http://ulmo.readthedocs.org/en/latest/>`ulmo`</a> package to retrieve U.S. Geological Survey surface water site data from the <a href=http://waterdata.usgs.gov/nwis>National Water Information System (NWIS) website</a>.  The function also puts the data from the website into a usable format. `download_usgs_site_info` gets site information from NWIS, which includes site name, watershed size, and coordinates.

In [ ]:
def download_usgs_site_data(siteno):
    # Download data
    data = ulmo.usgs.nwis.get_site_data(siteno, service="daily", period="all")
    
    # Parse values into dataframe
    values = pd.DataFrame(data['00060:00003']['values'])
    values['dates'] = pd.to_datetime(pd.Series(values['datetime']))
    values.set_index(['dates'], inplace=True)
    values[siteno] = values['value'].astype(float)
    values[str(siteno)+'qual'] = values['qualifiers']
    values = values.drop(['datetime','qualifiers','value'], axis=1)
    values = values.replace('-999999', np.NAN)
    values = values.dropna()
    
    # Parse site info into dataframe
    info = pd.DataFrame(data['00060:00003']['site'])
    info['latitude'] = info.loc['latitude', 'location']
    info['longitude'] = info.loc['longitude', 'location']
    info['latitude'] = info['latitude'].astype(float)
    info['longitude'] = info['longitude'].astype(float)
    info = info.drop(['default_tz', 'dst_tz', 'srs', 'uses_dst', 'longitude'], axis=0)
    info = info.drop(['agency', 'timezone_info', 'location', 'state_code', 'network'], axis=1)
    return info, values

`fdc` generates a flow duration curve for hydrologic data. A flow duration curve is a <a href=http://www.itl.nist.gov/div898/handbook/eda/section3/eda362.htm#PPF>percent point function (ppf)</a>, displaying discharge as a function of probability of that discharge occuring. The ppf is the inverse of the better known <a href=http://www.itl.nist.gov/div898/handbook/eda/section3/eda362.htm#CDF>cumulative distribution function (cdf)</a>. See <a href=http://pubs.usgs.gov/wsp/1542a/report.pdf>this USGS publication</a> for more information.

In [ ]:
def fdc(df, site, begyear=1900, endyear=2022, normalizer=1):
    '''
    Generate flow duration curve for hydrologic time series data
    
    PARAMETERS:
        df = pandas dataframe of interest; must have a date or date-time as the index
        site = pandas column containing discharge data; must be within df
        begyear = beginning year of analysis; defaults to 1900
        endyear = end year of analysis; defaults to 2022
        normalizer = value to use to normalize discharge; defaults to 1 (no normalization)
    
    RETURNS:
        probability, discharge
        
    REQUIRES:
        numpy as np
        pandas as pd
        matplotlib.pyplot as plt
        scipy.stats as sp
    '''
    # limit dataframe to only the site
    df = df[[site]]
    
    # filter dataframe to only include dates of interest
    dt_index = pd.to_datetime(df.index)
    data = df[(dt_index > dt.datetime(begyear, 1, 1))&(dt_index < dt.datetime(endyear, 1, 1))]

    # remove na values from dataframe
    data = data.dropna()

    # take average of each day of year (from 1 to 366) over the selected period of record
    data['doy'] = data.index.dayofyear
    dailyavg = data[site].groupby(data['doy']).mean()
        
    data = np.sort(dailyavg)

    ## uncomment the following to use normalized discharge instead of discharge
    #mean = np.mean(data)
    #std = np.std(data)
    #data = [(data[i]-np.mean(data))/np.std(data) for i in range(len(data))]
    data = [(data[i])/normalizer for i in range(len(data))]
    
    # ranks data from smallest to largest
    ranks = sp.rankdata(data, method='average')

    # reverses rank order
    ranks = ranks[::-1]
    
    # calculate probability of each rank
    prob = [(ranks[i]/(len(data)+1)) for i in range(len(data)) ]
    
    return prob, data

The `fdcmatch` uses Python's optimization capabilities to fit natural logarithim and exponential functions the flow duration curves.

In [ ]:
def fdcmatch(df, site, begyear=1900, endyear=2015, normalizer=1, fun=1):
    '''
    * This function creates a flow duration curve (or its inverse) and then matches a natural logrithmic function (or its inverse - exp) 
    to the flow duration curve
    * The flow duration curve will be framed for averaged daily data for the duration of one year (366 days)
    
    PARAMETERS:
        df = pandas dataframe of interest; must have a date or date-time as the index
        site = pandas column containing discharge data; must be within df
        begyear = beginning year of analysis; defaults to 1900
        endyear = end year of analysis; defaults to 2015
        normalizer = value to use to normalize discharge; defaults to 1 (no normalization)
        fun = 1 for probability as a function of discharge; 0 for discharge as a function of probability; default=1 
            * 1 will choose:
                prob = a*ln(discharge*b+c)+d
            * 0 will choose:
                discharge = a*exp(prob*b+c)+d
    RETURNS:
        para, parb, parc, pard, r_squared_value, stderr
    
        par = modifying variables for functions = a,b,c,d
        r_squared_value = r squared value for model
        stderr = standard error of the estimate
    
    REQUIREMENTS:
        pandas, scipy, numpy
    '''
    df = df[[site]]
    
    # filter dataframe to only include dates of interest
    dt_index = pd.to_datetime(df.index)
    data = df[(dt_index > dt.datetime(begyear, 1, 1))&(dt_index < dt.datetime(endyear, 1, 1))]

    # remove na values from dataframe
    data = data.dropna()

    # take average of each day of year (from 1 to 366) over the selected period of record
    data['doy']=data.index.dayofyear
    dailyavg = data[site].groupby(data['doy']).mean()
        
    data = np.sort(dailyavg)

    ## uncomment the following to use normalized discharge instead of discharge
    #mean = np.mean(data)
    #std = np.std(data)
    #data = [(data[i]-np.mean(data))/np.std(data) for i in range(len(data))]
    data = [(data[i])/normalizer for i in range(len(data))]
    
    # ranks data from smallest to largest
    ranks = sp.rankdata(data, method='average')

    # reverses rank order
    ranks = ranks[::-1]
    
    # calculate probability of each rank
    prob = [(ranks[i]/(len(data)+1)) for i in range(len(data)) ]
 
    # choose which function to use
    try:
        if fun == 1:
            # function to determine probability as a function of discharge
            def func(x, a, b, c, d):
                return a*np.log(x*b+c)+d

            # matches func to data
            par, cov = op.curve_fit(func, data, prob)

            # checks fit of curve match
            slope, interecept, r_value, p_value, stderr = \
            sp.linregress(prob, [par[0]*np.log(data[i]*par[1]+par[2])+par[3] for i in range(len(data))])
        else:
            # function to determine discharge as a function of probability
            def func(x, a, b, c, d):
                return a*np.exp(x*b+c)+d

            # matches func to data
            par, cov = op.curve_fit(func, prob, data)

            # checks fit of curve match
            slope, interecept, r_value, p_value, stderr = \
            sp.linregress(data, [par[0]*np.exp(prob[i]*par[1]+par[2])+par[3] for i in range(len(prob))])

        # return parameters (a,b,c,d), r-squared of model fit, and standard error of model fit 
        return par[0], par[1], par[2], par[3], round(r_value**2,2), round(stderr,5)
    except (RuntimeError, TypeError, ValueError):
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

## Download Data

The list `sites` designates the USGS surface sites you are interested in analyzing.

In [ ]:
sites = ['09309600','09309800', '09310000','09310500','09310700','09311500','09312700','09313000',
         '09317000','09317919','09317920','09317997','09318000','09318500','09319000',
         '09323000','09324000','09324200','09324500','09325000','09325100','09326500','09327500','09327550',
         '09330500','09331900','09331950','09332100', '10148500',
         '10205030','10206000','10206001','10208500',
         '10210000','10211000','10215700','10215900','10216210','10216400','10217000']
sitelab = ['Q' + site for site in sites]
print(len(sites))

In [ ]:
d = dict()
z = dict()
for site in sites:
    z[site], d[site] = download_usgs_site_data(site)

Merge data into a single dataframe so the data are aligned. Also, add day, month, and year columns to make summarizing the data more simple.

In [ ]:
f = dict()
f[sites[0]] = d[sites[0]]
for i in range(len(sites) - 1):
    f[sites[i + 1]] = pd.merge(d[sites[i + 1]], f[sites[i]], left_index=True, right_index=True, how='outer')

In [ ]:
USGS_Site_Data = f[sites[-1]]
USGS_Site_Data['mon'] = USGS_Site_Data.index.month
USGS_Site_Data['yr'] = USGS_Site_Data.index.year
USGS_Site_Data['dy'] = USGS_Site_Data.index.dayofyear

In [ ]:
USGS_Site_Info = pd.concat(z)
USGS_Site_Info = USGS_Site_Info.reset_index()
USGS_Site_Info = USGS_Site_Info.drop(['level_1'],axis=1)
USGS_Site_Info = USGS_Site_Info.set_index(['level_0'])
USGS_Site_Info = USGS_Site_Info.drop(['code'],axis=1)

Lets extract the measurement start and end dates from the station data, as well as some basic summary statistics. We can tack this information onto the site information table we created above.

In [ ]:
q = dict()
m = dict()
for site in sites:
    q[site] = USGS_Site_Data[site].first_valid_index()
    m[site] = USGS_Site_Data[site].last_valid_index()

USGS_start_date = pd.DataFrame(data=q, index=[0])
USGS_finish_date = pd.DataFrame(data=m, index=[0])
USGS_start_date = USGS_start_date.transpose()
USGS_start_date['start_date'] = USGS_start_date[0]
USGS_start_date = USGS_start_date.drop([0],axis=1)
USGS_finish_date = USGS_finish_date.transpose()
USGS_finish_date['fin_date'] = USGS_finish_date[0]
USGS_finish_date = USGS_finish_date.drop([0],axis=1)
USGS_start_fin = pd.merge(USGS_finish_date,USGS_start_date, left_index=True, right_index=True, how='outer')
USGS_Site_Info = pd.merge(USGS_start_fin,USGS_Site_Info, left_index=True, right_index=True, how='outer')
USGS_sum_stats = USGS_Site_Data[sites].describe()
USGS_sum_stats = USGS_sum_stats.transpose()
USGS_Site_Info = pd.merge(USGS_sum_stats,USGS_Site_Info, left_index=True, right_index=True, how='outer')

## Save Data

Save the data downloaded to CSV so it doesn't need to be downloaded again in the future.

In [ ]:
data_root = "/home/tethys/apps/awra_demo_2022/hydrology/"
USGS_Site_Data.to_csv(data_root + 'USGS_Site_Data.csv')
USGS_Site_Info.to_csv(data_root + 'USGS_Site_Info.csv')

## Load Data

Use this to load the data from CSV in the future.

In [ ]:
# Uncomment the lines below to load data from previously saved file
# data_root = "/home/tethys/apps/awra_demo_2022/hydrology/"
# USGS_Site_Data = pd.read_csv(data_root + 'USGS_Site_Data.csv', low_memory=False)
# USGS_Site_Info = pd.read_csv(data_root + 'USGS_Site_Info.csv')

## Visualization

The next two lines will display a summary of what is contained in the dataframes.

In [ ]:
USGS_Site_Data

In [ ]:
USGS_Site_Info

### Map Sites

Use Plotly's Scattermapbox module to render a simple map of the sites. Convert the DataFrame to a GeoDataFrame from geopandas for ease.

In [ ]:
fig = go.Figure(go.Scattermapbox(
    mode="markers",
    lon=USGS_Site_Info.longitude,
    lat=USGS_Site_Info.latitude,
    marker={'size': 10}
))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': -111, 'lat': 38.7},
        'style': "stamen-terrain",
        'zoom': 6
})

fig.show()

### Flow Duration Curves

This example produces the Flow Duration Curves for the given station. A flow duration curve is a <a href=http://www.itl.nist.gov/div898/handbook/eda/section3/eda362.htm#PPF>percent point function (ppf)</a>, displaying discharge as a function of probability of that discharge occuring. The ppf is the inverse of the better known <a href=http://www.itl.nist.gov/div898/handbook/eda/section3/eda362.htm#CDF>cumulative distribution function (cdf)</a>. See <a href=http://pubs.usgs.gov/wsp/1542a/report.pdf>this USGS publication</a> for more information.

In [ ]:
site_index = 2
site_no = USGS_Site_Info.index[site_index]
site_name = USGS_Site_Info['name'][site_index]
site_beg_record = str(USGS_Site_Info['start_date'][site_index])[0:10]
site_fin_record = str(USGS_Site_Info['fin_date'][site_index])[0:10]

# Calculate the flow duration curves
prob1, discharge1 = fdc(USGS_Site_Data, site_no, 1900, 2020)
prob2, discharge2 = fdc(USGS_Site_Data, site_no, 1900, 1970)
prob3, discharge3 = fdc(USGS_Site_Data, site_no, 1970, 2020)

# Plot with plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=prob1, y=discharge1, name=f'{site_no} 1900-2020'))
fig.add_trace(go.Scatter(x=prob2, y=discharge2, name=f'{site_no} 1900-1970'))
fig.add_trace(go.Scatter(x=prob3, y=discharge3, name=f'{site_no} 1970-2020'))

fig.update_layout(
    title=f"Flow duration curve for {site_name} ({site_no})<br>"
          f"Record: {site_beg_record} to {site_fin_record}",
    xaxis_title='probability that discharge was exceeded or equaled',
    xaxis_dtick=0.05,
    yaxis_title='discharge (cfs)',
    yaxis_type='log'
)

fig.show()

## Write to GeoJSON for Web

In [ ]:
USGS_Site_Info = gpd.GeoDataFrame(
    USGS_Site_Info, 
    geometry=gpd.points_from_xy(USGS_Site_Info.longitude, USGS_Site_Info.latitude)
)

In [ ]:
def serializer(o):
    return str(o)
    

USGS_Site_Info.to_json(default=serializer)

In [ ]:
with open('USGS_Site_Info.geojson', 'w') as f:
    f.write(USGS_Site_Info.to_json(default=serializer))